In [6]:
import pandas as pd
import numpy as np
from numpy import std, mean, sqrt, median
import matplotlib.pyplot as plt
from scipy.stats import expon, reciprocal
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from numpy.random import randint
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression 
import lightgbm as lgb
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Kkma, Okt  
import pickle
kkma = Kkma()
okt = Okt()

In [2]:
text_df = pd.read_csv('2019_news_article_data_with_video_label.csv', encoding='CP949')
text_df = text_df[['news_text', 'title', 'label']]
text_df["news_text"] = text_df['news_text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ,]","")
text_df['title'] = text_df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ,]","")

In [3]:
print(len(text_df))
text_df.head()

43377


,news_text,title,label
0,오전 시부터 방송 통해 신년사 발표우리 주동적 노력으로 조선반도에 평화기류 서울연합...,김정은 조선반도 항구적 평화지대로 만들려는 확고한 의지종합,20190101_100_1
1,국립극장서 출발팔각정서 의인참모들과 해맞이 서울연합뉴스 박경준 기자 문재인 대통령...,"문대통령, 의인 명과 분간 남산 등반 새해맞이",20190101_100_2
2,조선중앙 분간 신년사 녹화방송지난해까지 번의 신년사처럼강당 마이크 앞에 서 있는 대...,김정은 파격 변신 원고 들고 집무실 소파에 앉아 토크쇼 연출,20190101_100_3
3,이르면 오늘 중 유튜브 계정 게시할 듯서울뉴스 문대현 기자 한일 레이더 갈등과 관...,", 레이더 영상 공개에 맞대응반박 영상 제작",20190101_100_4
4,새해 월일 시분에 맞춰 녹화실세 인물 이례적으로 동반 출연내부 결속정상국가 이미지 ...,"김정은 신년사, 정상국가 이미지 강조김여정조용원 실세 재확인",20190101_100_5


In [4]:
label_df = pd.read_csv('groupby7th.csv')
label_df = pd.get_dummies(label_df, columns=['publisher', 'topic_cate'])
doc2vec_df = label_df.merge(text_df, on='label', how='left')
doc2vec_df = doc2vec_df[['title', 'news_text', 'label', 'cluster_7']]
doc2vec_df

,title,news_text,label,cluster_7
0,"문대통령, 의인 명과 분간 남산 등반 새해맞이",국립극장서 출발팔각정서 의인참모들과 해맞이 서울연합뉴스 박경준 기자 문재인 대통령...,20190101_100_2,6
1,김정은 파격 변신 원고 들고 집무실 소파에 앉아 토크쇼 연출,조선중앙 분간 신년사 녹화방송지난해까지 번의 신년사처럼강당 마이크 앞에 서 있는 대...,20190101_100_3,1
2,", 레이더 영상 공개에 맞대응반박 영상 제작",이르면 오늘 중 유튜브 계정 게시할 듯서울뉴스 문대현 기자 한일 레이더 갈등과 관...,20190101_100_4,3
3,"김정은 신년사, 정상국가 이미지 강조김여정조용원 실세 재확인",새해 월일 시분에 맞춰 녹화실세 인물 이례적으로 동반 출연내부 결속정상국가 이미지 ...,20190101_100_5,1
4,"대통령, 새해 인사 한분 한분 삶이 나아지도록 노력","따뜻히 세상 밝히라는 촛불 마음, 결코 잊지 않겠다서울뉴시스김태규 기자 문재인 대...",20190101_100_8,6
...,...,...,...,...
30851,환전빈집공유카풀 되는 게 뭐냐대통령 말에도 꿈쩍않는 규제,규제업계와 충돌 탓에끝내 사업 접는 스타트업들문 대통령 네거티브 강조했지만부처는 포...,20191103_105_9,6
30852,베일 벗은 디아블로오버워치블리즈컨서 첫 선,게임 커뮤니티 예상 적중출시 시점 미정아이뉴스 문영수 기자 전 세계 게임팬이 주목하...,20191103_105_12,2
30853,옆자리로 아베 이끈 대통령냉기류 녹인 분 즉석환담종합,"양국 사전조율 없었다대통령, 즉흥적으로 아베에 환담 권유일어 통역도 준비 못해단독...",20191104_100_1,6
30854,국감 파행 부른 버럭 참모들 서도 이러니 교체설 나와,"정의용, 위협 축소 잇단 발언에 안보실장 상황인식 너무 나이브와 충돌한 노영민강기...",20191104_100_2,4


## preprocessing 

In [7]:
with tf.device('/gpu:0'):
    # okt
    news_text_list_okt = [(okt.morphs(row.news_text), row.label) for i, row in doc2vec_df.iterrows()]
    title_list_okt = [(okt.morphs(row.title), row.label) for i, row in doc2vec_df.iterrows()]

    print("##"*20)
    print("tags and its texts")
    print("##"*20)

    for text, tags in news_text_list_okt[:1]:
        print(f"tags: {tags}, text: {text}")

    # save additional module
    with open('G:/내 드라이브/news_text_list_okt.data', 'wb') as f:
        pickle.dump(news_text_list_okt, f)

    with open('G:/내 드라이브/title_list_okt.data', 'wb') as f:
        pickle.dump(title_list_okt, f)

In [9]:
# load okt module
with open('G:/내 드라이브/news_text_list_okt.data', 'rb') as f:
    news_text = pickle.load(f)
with open('G:/내 드라이브/title_list_okt.data', 'rb') as f:
    news_title = pickle.load(f)

In [ ]:
# kkoma
news_text_list_kkoma = [(kkma.morphs(row.news_text), row.label) for i, row in doc2vec_df.iterrows()]
title_list_kkoma = [(kkma.morphs(row.title), row.label) for i, row in doc2vec_df.iterrows()]

print("##"*20)
print("tags and its texts")
print("##"*20)

for text, tags in news_text_list_kkoma[:1]:
    print(f"tags: {tags}, text: {text}")
    
# save additional module
with open('G:/내 드라이브/news_text_list_kkoma.data', 'wb') as f:
    pickle.dump(news_text_list_kkoma, f, protocol = pickle.highest_protocol)
    
with open('G:/내 드라이브/title_list_kkoma.data', 'wb') as f:
    pickle.dump(title_list_kkoma, f)

In [5]:
# load kkoma module
with open('G:/내 드라이브/news_text_list_kkoma.data', 'rb') as f:
    news_text = pickle.load(f)
with open('G:/내 드라이브/title_list_kkoma.data', 'rb') as f:
    news_title = pickle.load(f)

In [11]:
# 한 글자 제거
for morp_list, label in news_text: # 기사 본문
    for morp in morp_list:
        if len(morp) < 2:
            morp_list.remove(morp)
            
for morp_list, label in news_title: # 제목
    for morp in morp_list:
        if len(morp) < 2:
            morp_list.remove(morp)  

In [13]:
news_text_tagged_documents = [TaggedDocument(words=text, tags=[label]) for text, label in news_text]
news_title_tagged_documents = [TaggedDocument(words=text, tags=[label]) for text, label in news_title]
news_text_tagged_documents[:2]

[TaggedDocument(words=['국립', '극장', '출발', '팔각정', '의인', '참모', '해맞이', '서울', '연합', '뉴스', '기자', '재인', '대통령', '시민', '함께', '남산', '오르', '새해', '첫날', '맞이하', '대통령', '이날', '오전', '새해', '일정', '으로', '박재', '홍유', '동운', '박종훈', '안상', '세은', '현희', '빛내', '의인', '명과', '함께', '남산', '으로', '신년', '맞이', '해돋이', '산행', '다고', '청와', '대가', '밝히', '박재', '지난해', '서울', '봉천동', '원룸', '화재', '현장', '에서', '위험', '무릅쓰', '대학생', '구조', '화재', '초기', '진압', '유동', '지난해', '고창군', '에서', '으로', '추락', '승용차', '에서', '불이', '운전자', '구조', '박종훈', '지난해', '경북', '봉화군', '소천면', '사무소', '총기', '사건', '현장', '에서', '범인', '제압', '추가', '적인', '인명', '피해', '복지', '재단', '으로', '부터', '상금', '천만', '현장', '에서', '순직', '공무원', '유가족', '에게', '전달', '제주', '해경', '안상', '지난해', '제주', '우도', '에서', '발생', '천급', '유조선', '충돌', '사고', '선체', '에서', '쏟아지', '기름', '위하', '수중', '봉쇄', '작업', '벌이', '피해', '공로', '인정받', '경이', '선정', '해경', '최고', '영웅', '선정', '각각', '중학생', '고등학생', '세은', '현희', '지난해', '광주', '남구', '초등학교', '흘리', '쓰러지', '환자', '발견', '소방서', '구조', '요청', '구급차', '도착', '까지', '현장', '에서', '환자', '보호', '병원', '까지', 

In [14]:
vector_size = 30
news_title_model = Doc2Vec(documents = news_title_tagged_documents, vector_size = vector_size, dm=1, epochs=40, min_count=20, sample = .01, negative = 5)
news_text_model = Doc2Vec(documents = news_text_tagged_documents, vector_size = vector_size, dm=1, epochs=40, min_count=20, sample = .01, negative = 5)

In [15]:
news_title_dict_ = dict(zip(doc2vec_df['label'].to_list(), news_title_model.docvecs))
news_text_dict_ = dict(zip(doc2vec_df['label'].to_list(), news_text_model.docvecs))
columns = [f'doc2vec_f{i}' for i in range(1,vector_size+1)]
news_title_feature = pd.DataFrame.from_dict(news_title_dict_, orient='index', columns = columns).reset_index()
news_text_feature = pd.DataFrame.from_dict(news_text_dict_, orient='index', columns = columns).reset_index()
news_title_feature = news_title_feature.rename(columns={'index':'label'})
news_text_feature = news_text_feature.rename(columns={'index':'label'})
news_text_feature

,label,doc2vec_f1,doc2vec_f2,doc2vec_f3,doc2vec_f4,doc2vec_f5,doc2vec_f6,doc2vec_f7,doc2vec_f8,doc2vec_f9,...,doc2vec_f21,doc2vec_f22,doc2vec_f23,doc2vec_f24,doc2vec_f25,doc2vec_f26,doc2vec_f27,doc2vec_f28,doc2vec_f29,doc2vec_f30
0,20190101_100_2,-0.197901,0.490796,-2.063553,-1.910879,1.148224,0.956791,-0.648876,1.535135,-1.109166,...,1.592078,1.448688,0.544307,1.332618,1.244040,0.962417,1.728567,-1.284172,1.526530,-3.156833
1,20190101_100_3,-0.261700,1.296679,-3.012987,0.735239,-0.868942,-2.240913,0.040668,1.883911,-5.137534,...,-4.593400,-0.381968,3.286524,0.086809,0.728756,-1.776530,-0.533265,-1.777030,1.769237,1.396210
2,20190101_100_4,-1.541314,0.510345,2.047536,-1.187214,-2.316188,-1.181157,-1.291789,2.795046,-3.016417,...,-1.557484,-0.581565,-0.011952,-0.951095,0.394911,3.826283,-0.061555,-0.537328,0.735449,2.548155
3,20190101_100_5,-1.182282,3.432568,-3.638005,-0.230471,0.108589,-1.178796,-0.521124,1.607848,-5.780591,...,-2.862867,-1.248374,3.214693,0.751261,2.138492,-2.254322,-0.808608,-1.690833,2.837235,0.366293
4,20190101_100_8,-0.573823,1.292074,0.904056,-1.669159,0.505589,0.895228,0.063658,-0.374542,-2.067388,...,-2.343594,1.945336,1.456496,1.562109,1.854823,-0.966963,0.890183,-0.786168,1.266806,0.070791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30851,20191103_105_9,-2.621768,-0.159290,-1.602108,0.512224,-3.853437,0.593677,2.941544,-1.466924,-0.737685,...,-1.825592,1.330916,-0.971741,1.511242,1.856054,2.221554,1.472769,-1.488643,1.732934,-2.271831
30852,20191103_105_12,-1.630731,0.365030,-0.378121,-2.987555,0.651038,-0.335619,-2.438124,-5.145711,-1.423682,...,0.771561,0.675691,5.712713,2.505473,5.269382,2.085714,1.194146,-2.514995,4.033973,0.551535
30853,20191104_100_1,-5.506438,3.389144,2.478182,-1.057835,3.367210,-3.332856,1.862091,2.555987,-3.965681,...,-6.028215,3.399819,-2.335952,2.756252,1.439229,-0.820323,-0.397303,-0.311202,2.044544,0.468076
30854,20191104_100_2,1.670414,4.349333,2.753907,2.153308,-2.333617,-0.303283,2.405213,2.615259,-3.790950,...,-2.787119,-5.124445,0.141969,2.784440,-0.648818,-0.202630,-1.230288,-1.660625,1.968907,-3.719902


In [16]:
df_news_title = label_df.merge(news_title_feature, on='label', how='inner')
df_news_text = label_df.merge(news_text_feature, on='label', how='inner')
df_news_title

,Unnamed: 0,image_counts,video_counts,people_in_image_counts,picture_image_counts,graph_image_counts,title_length,main_length,title_scoop_binary,celebrity,...,doc2vec_f21,doc2vec_f22,doc2vec_f23,doc2vec_f24,doc2vec_f25,doc2vec_f26,doc2vec_f27,doc2vec_f28,doc2vec_f29,doc2vec_f30
0,0,2.0,1.0,43.0,2.0,0.0,14.0,383.0,0.0,14.0,...,0.029215,-0.141007,-0.074673,0.038926,-0.245232,-0.211187,-0.069065,0.011312,-0.155210,0.402694
1,1,2.0,0.0,7.0,2.0,0.0,12.0,739.0,0.0,4.0,...,-0.219160,-0.142037,-0.051788,0.250279,-0.087178,-0.025671,-0.155690,-0.038890,0.019556,-0.124459
2,2,1.0,0.0,0.0,1.0,0.0,14.0,330.0,0.0,1.0,...,-0.066739,-0.169182,0.258120,0.351970,0.047965,0.112163,-0.231540,-0.314172,0.095890,-0.469828
3,3,2.0,0.0,14.0,2.0,0.0,14.0,836.0,0.0,3.0,...,0.204239,0.097805,0.185986,-0.020486,0.470883,-0.205122,0.002773,-0.101065,0.029049,-0.060969
4,4,1.0,0.0,4.0,1.0,0.0,15.0,183.0,0.0,1.0,...,0.027347,-0.289223,-0.105566,0.107552,-0.053433,-0.106938,-0.132141,-0.042576,0.139377,0.009600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30851,30851,2.0,0.0,3.0,1.0,1.0,20.0,740.0,0.0,4.0,...,0.170481,0.382994,-0.030778,-0.392720,-0.136604,0.005258,0.213579,0.443536,-0.512655,1.189870
30852,30852,2.0,0.0,0.0,2.0,0.0,13.0,689.0,0.0,4.0,...,-0.212289,-0.070301,0.045315,0.189256,0.086311,0.106381,-0.082910,-0.024551,0.126721,-0.102705
30853,30853,2.0,0.0,7.0,2.0,0.0,18.0,1046.0,0.0,36.0,...,0.281945,-0.001369,0.067172,0.029269,-0.063315,-0.277693,0.279080,-0.749615,-0.384954,0.774650
30854,30854,1.0,0.0,4.0,1.0,0.0,17.0,792.0,0.0,19.0,...,0.042933,0.188369,-0.240451,0.075128,0.118195,0.068045,-0.042903,-0.037242,0.324202,-0.193169


In [21]:
df = df_news_text
X = df.drop(columns=['label', 'cluster_7', 'Unnamed: 0'])
y = df['cluster_7']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0, stratify = y)

columns_unscaling = []
for column in X.columns:
    if 'publisher' in column:
        columns_unscaling.append(column)
    elif 'topic_cate' in column:
        columns_unscaling.append(column)

In [22]:
# split columns for scaling
X_train_doc = X_train[X_train.columns.intersection(columns_unscaling)] # categorical features
X_test_doc = X_test[X_test.columns.intersection(columns_unscaling)]
X_train_for_scaling = X_train.drop(columns=columns_unscaling) # other features
X_test_for_scaling = X_test.drop(columns=columns_unscaling)

# scaling
full_pipeline = Pipeline([('scaler', StandardScaler())])
X_train_prepared = full_pipeline.fit_transform(X_train_for_scaling)
X_test_prepared = full_pipeline.transform(X_test_for_scaling)

# merge again
X_train_prepared = np.concatenate((X_train_prepared, X_train_doc.to_numpy()), axis=1)
X_test_prepared = np.concatenate((X_test_prepared, X_test_doc.to_numpy()), axis=1)

print(X_train_prepared.shape)
print(X_test_prepared.shape)

(21599, 123)
(9257, 123)


## 1. Logistic Regression 

In [140]:
param_distribs = {
    'C': reciprocal(10, 100000),
    'tol': expon(scale=.1),
    'penalty' : ['l1', 'l2'],
    'solver' : ['lbfgs', 'liblinear']
}
model = LogisticRegression()

grid = RandomizedSearchCV(model, param_distributions=param_distribs, scoring = 'accuracy', n_iter = 10, n_jobs=-1, cv=5)
grid.fit(X_train_prepared, y_train)

print('optimal train score: {:.3f}'.format(grid.best_score_))
print('test score: {:.3f}'.format(grid.score(X_test_prepared, y_test)))
print('optimal parameter: {}'.format(grid.best_params_))

optimal train score: 0.443
test score: 0.437
optimal parameter: {'tol': 0.0006951927961775605, 'solver': 'liblinear', 'penalty': 'l1', 'C': 0.08858667904100823}


In [9]:
final_model = grid.best_estimator_
print(final_model.score(X_test_prepared, y_test))

y_pred = final_model.predict(X_test_prepared)
print(classification_report(y_test, y_pred))

0.44031543696661984
              precision    recall  f1-score   support

           1       0.48      0.53      0.50      1627
           2       0.29      0.13      0.18      1825
           3       0.38      0.27      0.32      1878
           4       0.45      0.63      0.53      2467
           5       0.49      0.62      0.55      1460

    accuracy                           0.44      9257
   macro avg       0.42      0.44      0.42      9257
weighted avg       0.42      0.44      0.42      9257



## 2. SVC

In [31]:
param_distribs = {
    'C': reciprocal(0.0001, 10),
    'gamma': expon(scale=1),
    'kernel': ['linear', 'rbf', 'sigmoid']
}
model = SVC()

grid = RandomizedSearchCV(model, param_distributions=param_distribs, scoring = 'accuracy', n_iter = 10, n_jobs=-1, cv=5)
grid.fit(X_train_prepared, y_train)

print('optimal train score: {:.3f}'.format(grid.best_score_))
print('test score: {:.3f}'.format(grid.score(X_test_prepared, y_test)))
print('optimal parameter: {}'.format(grid.best_params_))

optimal train score: 0.380
test score: 0.386
optimal parameter: {'C': 1.0498444504452187, 'gamma': 2.697429308193368, 'kernel': 'linear'}
SVC(C=1.0498444504452187, gamma=2.697429308193368, kernel='linear')


In [32]:
final_model = grid.best_estimator_
print(final_model.score(X_test_prepared, y_test))

y_pred = final_model.predict(X_test_prepared)
print(classification_report(y_test, y_pred))

0.3857744653272845
              precision    recall  f1-score   support

           1       0.32      0.31      0.32      1081
           2       0.36      0.38      0.37       512
           3       0.24      0.02      0.03       642
           4       0.47      0.72      0.57      1140
           5       0.45      0.47      0.46       814
           6       0.33      0.47      0.39      1322
           7       0.35      0.02      0.04       661

    accuracy                           0.39      6172
   macro avg       0.36      0.34      0.31      6172
weighted avg       0.36      0.39      0.34      6172



## 3. RandomForest 

In [123]:
param_distribs = {
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(10, 100)),
    'max_features': ['auto', 'sqrt'],
    'max_leaf_nodes': list(range(1, 10)),
    'n_estimators': list(range(10, 1000)),
    'min_samples_split': list(range(1, 10)),
    'min_samples_leaf': list(range(1, 10)),
    'class_weight' : ['balanced', 'balanced_subsample']
}
model = RandomForestClassifier(warm_start = True)

grid = RandomizedSearchCV(model, param_distributions=param_distribs, scoring = 'accuracy', n_iter = 10, n_jobs=-1, cv=5)
grid.fit(X_train_prepared, y_train)

print('optimal train score: {:.3f}'.format(grid.best_score_))
print('test score: {:.3f}'.format(grid.score(X_test_prepared, y_test)))
print('optimal parameter: {}'.format(grid.best_params_))

optimal train score: 0.429
test score: 0.423
optimal parameter: {'n_estimators': 773, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_leaf_nodes': 8, 'max_features': 'auto', 'max_depth': 25, 'criterion': 'gini', 'class_weight': 'balanced'}


In [124]:
final_model = grid.best_estimator_
print(final_model.score(X_test_prepared, y_test))

y_pred = final_model.predict(X_test_prepared)
print(classification_report(y_test, y_pred))

0.4232472723344496
              precision    recall  f1-score   support

           1       0.43      0.67      0.52      1627
           2       0.35      0.02      0.03      1825
           3       0.33      0.32      0.33      1878
           4       0.50      0.44      0.47      2467
           5       0.42      0.75      0.54      1460

    accuracy                           0.42      9257
   macro avg       0.41      0.44      0.38      9257
weighted avg       0.41      0.42      0.37      9257



In [95]:
tmp1 = pd.DataFrame({'features': X.columns, 'importance': final_model.feature_importances_})
tmp1.sort_values(by='importance', ascending=False)

## 4. LightGBM 

In [23]:
param_distribs = {
    'boosting_type': ['gbdt', 'dart', 'rf'],
    'num_leaves': list(range(10,10000)),
    'learning_rate': reciprocal(.0001, 1000),
    'n_estimators': list(range(10, 10000)),
    'objective': ['multiclass'],
    'min_child_weight': reciprocal(.0001, 1000),
    'min_child_samples': list(range(1,10000)),
    'subsample_for_bin': list(range(1, 100000))
}
model = lgb.LGBMClassifier()

grid = RandomizedSearchCV(model, param_distributions=param_distribs, scoring = 'accuracy', n_iter = 10, cv=5)
grid.fit(X_train_prepared, y_train)

print('optimal train score: {:.3f}'.format(grid.best_score_))
print('test score: {:.3f}'.format(grid.score(X_test_prepared, y_test)))
print('optimal parameter: {}'.format(grid.best_params_))

optimal train score: 0.463
test score: 0.485
optimal parameter: {'boosting_type': 'dart', 'learning_rate': 0.4544226383717861, 'min_child_samples': 3792, 'min_child_weight': 0.10684994832525575, 'n_estimators': 4371, 'num_leaves': 8070, 'objective': 'multiclass', 'subsample_for_bin': 96645}


In [26]:
final_model = grid.best_estimator_
print(final_model.score(X_test_prepared, y_test))

y_pred = final_model.predict(X_test_prepared)
print(classification_report(y_test, y_pred))

0.4852544020741061
              precision    recall  f1-score   support

           1       0.46      0.52      0.49      1708
           2       0.53      0.46      0.49       780
           3       0.39      0.30      0.34       974
           4       0.56      0.61      0.58      1701
           5       0.59      0.59      0.59      1150
           6       0.44      0.48      0.46      1977
           7       0.40      0.29      0.34       967

    accuracy                           0.49      9257
   macro avg       0.48      0.46      0.47      9257
weighted avg       0.48      0.49      0.48      9257



In [100]:
X.corr()

,image_counts,video_counts,people_in_image_counts,picture_image_counts,graph_image_counts,title_length,main_length,title_scoop_binary,celebrity,follow_up,good_news_bad_news,publisher,topic_cate
image_counts,1.000000,0.081086,0.466693,0.873840,0.403369,0.060712,0.449786,0.102404,0.161686,0.000757,0.044627,-0.015143,-0.003616
video_counts,0.081086,1.000000,0.076042,0.087378,-0.003681,0.008877,-0.003106,-0.010156,0.016685,-0.032060,0.022869,0.055182,0.018853
people_in_image_counts,0.466693,0.076042,1.000000,0.529638,-0.046788,0.033199,0.201568,0.012761,0.204121,-0.016813,0.040529,0.022872,0.080397
picture_image_counts,0.873840,0.087378,0.529638,1.000000,-0.068570,0.072909,0.363181,0.071207,0.221398,0.003346,0.062511,-0.021865,0.009757
graph_image_counts,0.403369,-0.003681,-0.046788,-0.068570,1.000000,-0.009985,0.240954,0.076707,-0.092407,-0.008141,-0.030565,0.009565,-0.027449
title_length,0.060712,0.008877,0.033199,0.072909,-0.009985,1.000000,0.082829,0.199766,0.101865,0.012600,-0.009027,-0.010604,-0.009386
main_length,0.449786,-0.003106,0.201568,0.363181,0.240954,0.082829,1.000000,0.086049,0.403106,0.069674,0.051995,0.059964,0.047014
title_scoop_binary,0.102404,-0.010156,0.012761,0.071207,0.076707,0.199766,0.086049,1.000000,-0.014004,-0.034439,0.013196,-0.051507,-0.030972
celebrity,0.161686,0.016685,0.204121,0.221398,-0.092407,0.101865,0.403106,-0.014004,1.000000,0.065070,0.053337,0.019327,0.144319
follow_up,0.000757,-0.032060,-0.016813,0.003346,-0.008141,0.012600,0.069674,-0.034439,0.065070,1.000000,0.004805,-0.048061,-0.008835


In [99]:
tmp1 = pd.DataFrame({'features': X.columns, 'importance': final_model.feature_importances_})
tmp1.sort_values(by='importance', ascending=False)

,features,importance
6,main_length,75924
11,publisher,75624
12,topic_cate,73424
9,follow_up,68969
5,title_length,48594
2,people_in_image_counts,43221
8,celebrity,41634
0,image_counts,12553
3,picture_image_counts,11007
4,graph_image_counts,6729


## 5. DNN 

In [ ]:
from keras.layers import Activation, BatchNormalization

In [178]:
def get_model():
    model = models.Sequential()
    model.add(Dense(512, activation = 'relu', input_shape = (93,)))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormlization())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(7, activation = 'softmax'))
    model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [188]:
from keras import models
from keras import layers
from keras.utils import to_categorical


model = get_model()
train_labels = to_categorical(y_train)
train_labels = train_labels[:, 1:]

X_val = X_train_prepared[:4000]
partial_X_train = X_train_prepared[4000:]

y_val = train_labels[:4000]
partial_y_train = train_labels[4000:]

history = model.fit(partial_X_train, partial_y_train, epochs = 10, batch_size = 64, validation_data = (X_val, y_val)) 

Train on 20684 samples, validate on 4000 samples
Epoch 1/10
20684/20684 [==============================] - 2s 83us/step - loss: 1.6563 - accuracy: 0.3516 - val_loss: 1.5830 - val_accuracy: 0.3862
Epoch 2/10
20684/20684 [==============================] - 2s 78us/step - loss: 1.5605 - accuracy: 0.3816 - val_loss: 1.5567 - val_accuracy: 0.3870
Epoch 3/10
20684/20684 [==============================] - 2s 74us/step - loss: 1.5336 - accuracy: 0.3975 - val_loss: 1.5549 - val_accuracy: 0.3902
Epoch 4/10
20684/20684 [==============================] - 2s 74us/step - loss: 1.5152 - accuracy: 0.4066 - val_loss: 1.5419 - val_accuracy: 0.3925
Epoch 5/10
20684/20684 [==============================] - 2s 74us/step - loss: 1.4999 - accuracy: 0.4118 - val_loss: 1.5305 - val_accuracy: 0.4027
Epoch 6/10
20684/20684 [==============================] - 2s 78us/step - loss: 1.4839 - accuracy: 0.4161 - val_loss: 1.5496 - val_accuracy: 0.3918
Epoch 7/10
20684/20684 [==============================] - 2s 75us/ste

In [189]:
test_labels = to_categorical(y_test)
test_labels = test_labels[:, 1:]

results = model.evaluate(X_test_prepared, test_labels)
results

6172/6172 [==============================] - 0s 17us/step


[1.5574280952990094, 0.3992222845554352]

In [190]:
y_test

18815    5
12113    5
6055     4
17598    4
24163    5
        ..
2327     7
19099    6
3173     3
15754    4
14227    1
Name: cluster_7, Length: 6172, dtype: int64

In [195]:
df_describe = pd.DataFrame(y_pred)
df_describe

,0
0,1
1,1
2,7
3,4
4,4
...,...
6167,7
6168,6
6169,1
6170,5


In [193]:
for i, row in df_describe.iterrows():
    if row[0] == 0:
        row[0] = 1
    elif row[0] == 1:
        row[0] = 2
    elif row[0] == 2:
        row[0] = 3
    elif row[0] == 3:
        row[0] = 4
    elif row[0] == 4:
        row[0] = 5
    elif row[0] == 5:
        row[0] = 6
    elif row[0] == 6:
        row[0] = 7

In [194]:
df_describe = df_describe[0].values

In [196]:
y_pred = model.predict_classes(X_test_prepared)
print(classification_report(y_test, df_describe))

              precision    recall  f1-score   support

           1       0.36      0.33      0.35      1081
           2       0.32      0.45      0.38       512
           3       0.25      0.10      0.15       642
           4       0.49      0.69      0.58      1140
           5       0.48      0.40      0.44       814
           6       0.35      0.48      0.40      1322
           7       0.39      0.04      0.07       661

    accuracy                           0.39      6172
   macro avg       0.38      0.36      0.34      6172
weighted avg       0.39      0.39      0.36      6172

